In [ ]:
## 프로그래스바 추가

##아웃룩 메일 아카이브

In [39]:
import win32com.client
import os, re
from datetime import datetime, timedelta
import base64
import uuid
import tkinter as tk
from tkinter import ttk
from tkcalendar import DateEntry  # 날짜 선택을 위한 캘린더 위젯 추가
import logging  # 로깅을 위한 모듈 추가
import time     # 실행시간 측정을 위한 모듈 추가

# ──────────────────────────────────────────────
# 1. 설정
# ──────────────────────────────────────────────
BASE_DIR = os.getcwd()
FOLDER_MAP = {"inbox": 6, "sent": 5}        # Outlook 기본 폴더 ID
DATE_FMT_FOLDER = "%Y%m%d"                  # yyyymmdd
DATE_FMT_MONTH  = "%y%m"                    # ⇦ ❶ 추가: yymm  (예: 2506)

# 로그 설정
LOG_FILE = os.path.join(BASE_DIR, f"outlook_backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log")
logging.basicConfig(
    filename=LOG_FILE,
    level=logging.INFO,
    format='%(asctime)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

INVALID_CHARS = re.compile(r'[\/:*?"<>|\x00-\x1F]')

RESERVED_NAMES = {
    "CON","PRN","AUX","NUL",
    *(f"COM{i}" for i in range(1,10)),
    *(f"LPT{i}" for i in range(1,10)),
}

def sanitize(text: str, max_len: int = 150) -> str:
    """
    Windows-safe 폴더/파일명으로 변환
      • 제어문자·금지문자 → '_' 치환
      • 끝 공백/마침표 제거
      • 예약어 회피
      • 길이 제한
    """
    cleaned = INVALID_CHARS.sub("_", text or "").strip()
    cleaned = cleaned.rstrip(". ")

    if cleaned.upper() in RESERVED_NAMES:
        cleaned = f"_{cleaned}"

    if len(cleaned) > max_len:
        cleaned = cleaned[:max_len]

    return cleaned or "untitled"

# 전역변수 선언
folder_key = ""
start_date = None
end_date = None
mail_count = {"inbox": 0, "sent": 0}  # 메일 카운트를 위한 변수 추가
total_mails = 0  # 전체 메일 수를 저장할 변수 추가
processed_mails = 0  # 처리된 메일 수를 저장할 변수 추가

def has_attachments(mail):
    """
    메일에 첨부파일이 있는지 확인하는 함수
    보낸편지함의 경우도 정상적으로 체크되도록 수정
    """
    try:
        if folder_key == "sent":
            return len(mail.Attachments) > 0
        return mail.Attachments.Count > 0
    except Exception:
        return False

def is_meeting_item(mail):
    try:
        if getattr(mail, "MeetingStatus", 0) != 0:
            return True
        return str(getattr(mail, "MessageClass", "")).startswith("IPM.Schedule.Meeting")
    except Exception:
        return False

def save_attachments(mail, target_dir):
    """
    첨부파일 저장 함수
    보낸편지함의 경우도 정상적으로 저장되도록 수정
    """
    try:
        if folder_key == "sent":
            for att in mail.Attachments:
                att.SaveAsFile(os.path.join(target_dir, sanitize(att.FileName)))
        else:
            for i in range(1, mail.Attachments.Count + 1):
                att = mail.Attachments.Item(i)
                att.SaveAsFile(os.path.join(target_dir, sanitize(att.FileName)))
    except Exception as e:
        logging.error(f"첨부파일 저장 오류: {e}")

def extract_inline_images(html_content, mail_dir):
    """
    HTML 내용에서 인라인 이미지를 추출하고 저장하는 함수
    """
    img_pattern = re.compile(r'<img[^>]+src="cid:([^"]+)"[^>]*>')
    
    def replace_image(match):
        cid = match.group(1)
        try:
            for att in (mail.Attachments if folder_key == "sent" else mail.Attachments):
                if str(att.PropertyAccessor.GetProperty("http://schemas.microsoft.com/mapi/proptag/0x3712001F")) == cid:
                    img_filename = f"img_{uuid.uuid4().hex[:8]}_{sanitize(att.FileName)}"
                    img_path = os.path.join(mail_dir, img_filename)
                    att.SaveAsFile(img_path)
                    return f'<img src="{img_filename}"'
        except:
            pass
        return match.group(0)
    
    return img_pattern.sub(replace_image, html_content)

def build_email_html(mail) -> str:
    """
    이메일 내용을 HTML 형식으로 구성
    인라인 이미지를 포함하여 저장
    """
    html_content = f"""
    <html>
    <head>
        <meta charset="utf-8">
        <style>
            body {{ font-family: Arial, sans-serif; }}
            .header {{ margin-bottom: 20px; }}
            .header-item {{ margin: 5px 0; }}
            .divider {{ border-top: 1px solid #ccc; margin: 10px 0; }}
            .body {{ white-space: pre-wrap; }}
            img {{ max-width: 100%; }}
        </style>
    </head>
    <body>
        <div class="header">
            <div class="header-item"><strong>From:</strong> {getattr(mail, 'SenderName', '')}</div>
            <div class="header-item"><strong>To:</strong> {getattr(mail, 'To', '')}</div>
            <div class="header-item"><strong>CC:</strong> {getattr(mail, 'CC', '')}</div>
            <div class="header-item"><strong>Subject:</strong> {getattr(mail, 'Subject', '')}</div>
            <div class="header-item"><strong>Date:</strong> {getattr(mail, 'SentOn', getattr(mail,'ReceivedTime', ''))}</div>
        </div>
        <div class="divider"></div>
        <div class="body">{getattr(mail, 'HTMLBody', getattr(mail, 'Body', ''))}</div>
    </body>
    </html>
    """
    return html_content

def is_in_date_range(dt):
    """
    메일 날짜가 선택된 날짜 범위 안에 있는지 확인하는 함수
    """
    if dt and start_date and end_date:
        mail_date = dt.date()
        return start_date <= mail_date <= end_date
    return False

outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

def count_total_mails():
    """
    선택된 날짜 범위 내의 전체 메일 수를 계산하는 함수
    """
    global total_mails
    total_mails = 0
    
    for folder_name in ["inbox", "sent"]:
        try:
            folder = outlook.GetDefaultFolder(FOLDER_MAP[folder_name])
            items = folder.Items
            items.Sort("[ReceivedTime]", True)
            
            # 메일 수를 정확하게 계산하기 위해 실제 처리할 메일만 카운트
            for mail in items:
                try:
                    if is_meeting_item(mail):
                        continue
                        
                    mail_dt = getattr(mail, "ReceivedTime", getattr(mail, "SentOn", None))
                    if not mail_dt:
                        continue
                        
                    if not is_in_date_range(mail_dt):
                        if mail_dt.date() < start_date:
                            break
                        continue
                    
                    # 이미 백업된 메일은 제외
                    month_dir = os.path.join(BASE_DIR, "받은편지" if folder_name == "inbox" else "보낸편지")
                    month_dir = os.path.join(month_dir, mail_dt.strftime(DATE_FMT_MONTH))
                    
                    if folder_name == "inbox":
                        counter = sanitize(getattr(mail, "SenderName", "unknown"))
                    else:
                        first_recipient = sanitize(getattr(mail, "To", "unknown").split(";")[0].split(",")[0])
                        counter = first_recipient or "unknown"
                        
                    subject = sanitize(getattr(mail, "Subject", "no_subject"))
                    attach_flag = "_첨부" if has_attachments(mail) else ""
                    mail_folder_name = f"{mail_dt.strftime(DATE_FMT_FOLDER)}_{counter}_{subject}{attach_flag}"
                    mail_dir = os.path.join(month_dir, mail_folder_name)
                    
                    if not os.path.exists(mail_dir):
                        total_mails += 1
                        
                except Exception as e:
                    logging.error(f"메일 카운트 중 오류: {str(e)}")
                    continue
                    
        except Exception as e:
            logging.error(f"{folder_name} 폴더 접근 중 오류: {str(e)}")
            continue

def process_mail_items(items, root_dir):
    """
    메일 아이템들을 처리하는 함수
    """
    global mail, processed_mails
    for mail in items:
        try:
            mail_dt = getattr(mail, "ReceivedTime", getattr(mail, "SentOn", None))
            if not is_in_date_range(mail_dt):
                if mail_dt and mail_dt.date() < start_date:
                    break
                continue
            if is_meeting_item(mail):
                continue

            month_dir = os.path.join(root_dir, mail_dt.strftime(DATE_FMT_MONTH))
            os.makedirs(month_dir, exist_ok=True)

            date_str = mail_dt.strftime(DATE_FMT_FOLDER)
            if folder_key == "inbox":
                counter = sanitize(getattr(mail, "SenderName", "unknown"))
            else:
                first_recipient = sanitize(getattr(mail, "To", "unknown").split(";")[0].split(",")[0])
                counter = first_recipient or "unknown"

            subject = sanitize(getattr(mail, "Subject", "no_subject"))
            attach_flag = "_첨부" if has_attachments(mail) else ""
            mail_folder_name = f"{date_str}_{counter}_{subject}{attach_flag}"

            mail_dir = os.path.join(month_dir, mail_folder_name)
            if os.path.exists(mail_dir):
                continue
            os.makedirs(mail_dir, exist_ok=True)

            htm_filename = f"{date_str}_{subject}.htm"
            html_content = build_email_html(mail)
            html_content = extract_inline_images(html_content, mail_dir)
            
            with open(os.path.join(mail_dir, htm_filename), "w", encoding="utf-8", errors="ignore") as f:
                f.write(html_content)

            if has_attachments(mail):
                save_attachments(mail, mail_dir)
            
            mail_count[folder_key] += 1  # 메일 카운트 증가
            processed_mails += 1  # 처리된 메일 수 증가
            
            # 프로그레스바 업데이트
            progress = (processed_mails / total_mails) * 100
            progress_var.set(progress)
            progress_label.config(text=f"진행률: {progress:.1f}% ({processed_mails}/{total_mails})")
            root.update()

        except Exception as e:
            logging.error(f"메일 처리 오류: {e}")

def process_folder(folder_name: str, root_name: str):
    """
    폴더와 그 하위 폴더의 메일을 처리하는 함수
    """
    global folder_key
    folder_key = folder_name

    root_dir = os.path.join(BASE_DIR, root_name)
    os.makedirs(root_dir, exist_ok=True)

    def process_subfolder(outlook_folder):
        items = outlook_folder.Items
        items.Sort("[ReceivedTime]", True)
        process_mail_items(items, root_dir)

        for subfolder in outlook_folder.Folders:
            process_subfolder(subfolder)

    root_folder = outlook.GetDefaultFolder(FOLDER_MAP[folder_key])
    process_subfolder(root_folder)

def set_date_range(months):
    """
    날짜 범위를 설정하는 함수
    """
    end_date_cal.set_date(datetime.now())
    start_date_cal.set_date(datetime.now() - timedelta(days=30*months))

def start_backup():
    """
    백업 시작 함수
    """
    global start_date, end_date, processed_mails, mail_count
    
    # 캘린더에서 선택된 날짜 가져오기
    start_date = start_date_cal.get_date()
    end_date = end_date_cal.get_date()
    
    # 날짜 범위 확인
    if start_date > end_date:
        status_label.config(text="시작 날짜가 종료 날짜보다 늦을 수 없습니다.")
        return
        
    if end_date > datetime.now().date():
        status_label.config(text="종료 날짜는 오늘 이후로 설정할 수 없습니다.")
        return
    
    # 변수 초기화
    mail_count = {"inbox": 0, "sent": 0}
    processed_mails = 0
    progress_var.set(0)  # 프로그레스바 초기화
    progress_label.config(text="")  # 진행률 레이블 초기화
    
    # 로그 시작
    logging.info(f"백업 시작")
    logging.info(f"시작 날짜: {start_date}")
    logging.info(f"종료 날짜: {end_date}")
    
    status_label.config(text="백업 진행 중...")
    root.update()
    
    start_time = time.time()  # 시작 시간 기록
    
    try:
        # 전체 메일 수 계산
        count_total_mails()
        
        if total_mails == 0:
            status_label.config(text="선택된 기간에 백업할 메일이 없습니다.")
            return
            
        process_folder("inbox", "받은편지")
        process_folder("sent", "보낸편지")
        
        end_time = time.time()  # 종료 시간 기록
        elapsed_time = end_time - start_time  # 실행 시간 계산
        
        # 결과 로깅
        logging.info(f"백업 완료")
        logging.info(f"실행 시간: {elapsed_time:.1f}초")
        logging.info(f"받은편지함 저장 개수: {mail_count['inbox']}개")
        logging.info(f"보낸편지함 저장 개수: {mail_count['sent']}개")
        logging.info(f"전체 저장 개수: {mail_count['inbox'] + mail_count['sent']}개")
        
        status_label.config(text="백업이 완료되었습니다!")
    except Exception as e:
        logging.error(f"백업 중 오류 발생: {str(e)}")
        status_label.config(text=f"오류 발생: {str(e)}")

# GUI 생성
root = tk.Tk()
root.title("아웃룩 메일 백업")
root.geometry("400x500")  # 창 크기 증가

# 프레임 생성 및 중앙 정렬
frame = ttk.Frame(root, padding="10")
frame.place(relx=0.5, rely=0.5, anchor="center")

# 날짜 범위 버튼 프레임
date_range_frame = ttk.Frame(frame)
date_range_frame.pack(pady=10)

# 날짜 범위 버튼들
ttk.Button(date_range_frame, text="최근 1개월", command=lambda: set_date_range(1)).pack(side=tk.LEFT, padx=5)
ttk.Button(date_range_frame, text="최근 3개월", command=lambda: set_date_range(3)).pack(side=tk.LEFT, padx=5)
ttk.Button(date_range_frame, text="최근 6개월", command=lambda: set_date_range(6)).pack(side=tk.LEFT, padx=5)
ttk.Button(date_range_frame, text="최근 1년", command=lambda: set_date_range(12)).pack(side=tk.LEFT, padx=5)

# 시작 날짜 선택
date_frame = ttk.Frame(frame)
date_frame.pack(pady=10)

ttk.Label(date_frame, text="시작 날짜:").pack()
start_date_cal = DateEntry(date_frame, width=15, background='darkblue',
                          foreground='white', borderwidth=2,
                          date_pattern='yyyy-mm-dd',
                          firstweekday='sunday')
start_date_cal.pack(pady=5)
start_date_cal.set_date(datetime.now() - timedelta(days=7))

# 종료 날짜 선택
ttk.Label(date_frame, text="종료 날짜:").pack()
end_date_cal = DateEntry(date_frame, width=15, background='darkblue',
                        foreground='white', borderwidth=2,
                        date_pattern='yyyy-mm-dd',
                        firstweekday='sunday')
end_date_cal.pack(pady=5)
end_date_cal.set_date(datetime.now())

# 백업 시작 버튼
backup_btn = ttk.Button(frame, text="백업 시작", command=start_backup)
backup_btn.pack(pady=20)

# 프로그레스바 추가
progress_var = tk.DoubleVar()
progress_bar = ttk.Progressbar(frame, length=300, mode='determinate', variable=progress_var)
progress_bar.pack(pady=10)

# 진행률 표시 레이블
progress_label = ttk.Label(frame, text="")
progress_label.pack(pady=5)

# 상태 표시 레이블
status_label = ttk.Label(frame, text="")
status_label.pack(pady=5)

if __name__ == "__main__":
    root.mainloop()


## 작업중

In [41]:
import win32com.client
import os, re
from datetime import datetime, timedelta
import base64
import uuid
import tkinter as tk
from tkinter import ttk
from tkcalendar import DateEntry  # 날짜 선택을 위한 캘린더 위젯 추가
import logging  # 로깅을 위한 모듈 추가
import time     # 실행시간 측정을 위한 모듈 추가

# ──────────────────────────────────────────────
# 1. 설정
# ──────────────────────────────────────────────
BASE_DIR = os.getcwd()
FOLDER_MAP = {"inbox": 6, "sent": 5}        # Outlook 기본 폴더 ID
DATE_FMT_FOLDER = "%Y%m%d"                  # yyyymmdd
DATE_FMT_MONTH  = "%y%m"                    # ⇦ ❶ 추가: yymm  (예: 2506)

# 로그 설정
LOG_FILE = os.path.join(BASE_DIR, f"outlook_backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log")
logging.basicConfig(
    filename=LOG_FILE,
    level=logging.INFO,
    format='%(asctime)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# 파일명에서 제거할 문자들 정규식 패턴 강화
INVALID_CHARS = re.compile(r'[\\/:*?"<>|]|\s+|[\x00-\x1F\x7F-\x9F]|[\u2000-\u200F\u2028-\u202F]')

RESERVED_NAMES = {
    "CON","PRN","AUX","NUL",
    *(f"COM{i}" for i in range(1,10)),
    *(f"LPT{i}" for i in range(1,10)),
}

def sanitize(text: str, max_len: int = 50) -> str:  # max_len을 50으로 줄임
    """
    Windows-safe 폴더/파일명으로 변환
    - 제어문자, 특수문자, 유니코드 공백문자 제거
    - 연속된 공백을 하나로 치환
    - 파일명 길이 제한 강화 (50자)
    - 한글 인코딩 문제 해결을 위한 정규화
    """
    if not text:
        return "untitled"
        
    # 유니코드 정규화 (한글 호환성)
    import unicodedata
    text = unicodedata.normalize('NFKC', text)
    
    # 특수문자 및 제어문자 제거
    cleaned = INVALID_CHARS.sub("_", text)
    cleaned = cleaned.strip(". _")  # 시작/끝의 점과 공백 제거
    
    # 연속된 언더스코어 제거
    cleaned = re.sub(r'_+', '_', cleaned)
    
    if cleaned.upper() in RESERVED_NAMES:
        cleaned = f"_{cleaned}"
        
    # 길이 제한 (경로 길이 문제 방지)
    if len(cleaned) > max_len:
        cleaned = cleaned[:max_len]
        
    return cleaned or "untitled"

# 전역변수 선언
folder_key = ""
start_date = None
end_date = None
mail_count = {"inbox": 0, "sent": 0}  # 메일 카운트를 위한 변수 추가
total_mails = 0  # 전체 메일 수를 저장할 변수 추가
processed_mails = 0  # 처리된 메일 수를 저장할 변수 추가

def has_attachments(mail):
    """
    메일에 첨부파일이 있는지 확인하는 함수
    보낸편지함의 경우도 정상적으로 체크되도록 수정
    """
    try:
        if folder_key == "sent":
            return len(mail.Attachments) > 0
        return mail.Attachments.Count > 0
    except Exception:
        return False

def is_meeting_item(mail):
    try:
        if getattr(mail, "MeetingStatus", 0) != 0:
            return True
        return str(getattr(mail, "MessageClass", "")).startswith("IPM.Schedule.Meeting")
    except Exception:
        return False

def save_attachments(mail, target_dir):
    """
    첨부파일 저장 함수
    보낸편지함의 경우도 정상적으로 저장되도록 수정
    """
    try:
        if folder_key == "sent":
            for att in mail.Attachments:
                safe_filename = sanitize(att.FileName)
                att.SaveAsFile(os.path.join(target_dir, safe_filename))
        else:
            for i in range(1, mail.Attachments.Count + 1):
                att = mail.Attachments.Item(i)
                safe_filename = sanitize(att.FileName)
                att.SaveAsFile(os.path.join(target_dir, safe_filename))
    except Exception as e:
        logging.error(f"첨부파일 저장 오류: {e}")

def extract_inline_images(html_content, mail_dir):
    """
    HTML 내용에서 인라인 이미지를 추출하고 저장하는 함수
    """
    img_pattern = re.compile(r'<img[^>]+src="cid:([^"]+)"[^>]*>')
    
    def replace_image(match):
        cid = match.group(1)
        try:
            for att in (mail.Attachments if folder_key == "sent" else mail.Attachments):
                if str(att.PropertyAccessor.GetProperty("http://schemas.microsoft.com/mapi/proptag/0x3712001F")) == cid:
                    img_filename = f"img_{uuid.uuid4().hex[:8]}_{sanitize(att.FileName)}"
                    img_path = os.path.join(mail_dir, img_filename)
                    att.SaveAsFile(img_path)
                    return f'<img src="{img_filename}"'
        except:
            pass
        return match.group(0)
    
    return img_pattern.sub(replace_image, html_content)

def build_email_html(mail) -> str:
    """
    이메일 내용을 HTML 형식으로 구성
    인라인 이미지를 포함하여 저장
    """
    html_content = f"""
    <html>
    <head>
        <meta charset="utf-8">
        <style>
            body {{ font-family: Arial, sans-serif; }}
            .header {{ margin-bottom: 20px; }}
            .header-item {{ margin: 5px 0; }}
            .divider {{ border-top: 1px solid #ccc; margin: 10px 0; }}
            .body {{ white-space: pre-wrap; }}
            img {{ max-width: 100%; }}
        </style>
    </head>
    <body>
        <div class="header">
            <div class="header-item"><strong>From:</strong> {getattr(mail, 'SenderName', '')}</div>
            <div class="header-item"><strong>To:</strong> {getattr(mail, 'To', '')}</div>
            <div class="header-item"><strong>CC:</strong> {getattr(mail, 'CC', '')}</div>
            <div class="header-item"><strong>Subject:</strong> {getattr(mail, 'Subject', '')}</div>
            <div class="header-item"><strong>Date:</strong> {getattr(mail, 'SentOn', getattr(mail,'ReceivedTime', ''))}</div>
        </div>
        <div class="divider"></div>
        <div class="body">{getattr(mail, 'HTMLBody', getattr(mail, 'Body', ''))}</div>
    </body>
    </html>
    """
    return html_content

def is_in_date_range(dt):
    """
    메일 날짜가 선택된 날짜 범위 안에 있는지 확인하는 함수
    """
    if dt and start_date and end_date:
        mail_date = dt.date()
        return start_date <= mail_date <= end_date
    return False

outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

def count_total_mails():
    """
    선택된 날짜 범위 내의 전체 메일 수를 계산하는 함수
    """
    global total_mails
    total_mails = 0
    
    for folder_name in ["inbox", "sent"]:
        try:
            folder = outlook.GetDefaultFolder(FOLDER_MAP[folder_name])
            items = folder.Items
            items.Sort("[ReceivedTime]", True)
            
            # 메일 수를 정확하게 계산하기 위해 실제 처리할 메일만 카운트
            for mail in items:
                try:
                    if is_meeting_item(mail):
                        continue
                        
                    mail_dt = getattr(mail, "ReceivedTime", getattr(mail, "SentOn", None))
                    if not mail_dt:
                        continue
                        
                    if not is_in_date_range(mail_dt):
                        if mail_dt.date() < start_date:
                            break
                        continue
                    
                    # 이미 백업된 메일은 제외
                    month_dir = os.path.join(BASE_DIR, "받은편지" if folder_name == "inbox" else "보낸편지")
                    month_dir = os.path.join(month_dir, mail_dt.strftime(DATE_FMT_MONTH))
                    
                    if folder_name == "inbox":
                        counter = sanitize(getattr(mail, "SenderName", "unknown"))
                    else:
                        first_recipient = sanitize(getattr(mail, "To", "unknown").split(";")[0].split(",")[0])
                        counter = first_recipient or "unknown"
                        
                    subject = sanitize(getattr(mail, "Subject", "no_subject"))
                    attach_flag = "_첨부" if has_attachments(mail) else ""
                    mail_folder_name = f"{mail_dt.strftime(DATE_FMT_FOLDER)}_{counter}_{subject}{attach_flag}"
                    mail_dir = os.path.join(month_dir, mail_folder_name)
                    
                    if not os.path.exists(mail_dir):
                        total_mails += 1
                        
                except Exception as e:
                    logging.error(f"메일 카운트 중 오류: {str(e)}")
                    continue
                    
        except Exception as e:
            logging.error(f"{folder_name} 폴더 접근 중 오류: {str(e)}")
            continue

def process_mail_items(items, root_dir):
    """
    메일 아이템들을 처리하는 함수
    """
    global mail, processed_mails
    for mail in items:
        try:
            mail_dt = getattr(mail, "ReceivedTime", getattr(mail, "SentOn", None))
            if not is_in_date_range(mail_dt):
                if mail_dt and mail_dt.date() < start_date:
                    break
                continue
            if is_meeting_item(mail):
                continue

            month_dir = os.path.join(root_dir, mail_dt.strftime(DATE_FMT_MONTH))
            os.makedirs(month_dir, exist_ok=True)

            date_str = mail_dt.strftime(DATE_FMT_FOLDER)
            if folder_key == "inbox":
                counter = sanitize(getattr(mail, "SenderName", "unknown"))
            else:
                first_recipient = sanitize(getattr(mail, "To", "unknown").split(";")[0].split(",")[0])
                counter = first_recipient or "unknown"

            subject = sanitize(getattr(mail, "Subject", "no_subject"))
            attach_flag = "_첨부" if has_attachments(mail) else ""
            mail_folder_name = f"{date_str}_{counter}_{subject}{attach_flag}"

            mail_dir = os.path.join(month_dir, mail_folder_name)
            if os.path.exists(mail_dir):
                continue
            os.makedirs(mail_dir, exist_ok=True)

            htm_filename = f"{date_str}_{subject}.htm"
            html_content = build_email_html(mail)
            html_content = extract_inline_images(html_content, mail_dir)
            
            with open(os.path.join(mail_dir, htm_filename), "w", encoding="utf-8", errors="ignore") as f:
                f.write(html_content)

            if has_attachments(mail):
                save_attachments(mail, mail_dir)
            
            mail_count[folder_key] += 1  # 메일 카운트 증가
            processed_mails += 1  # 처리된 메일 수 증가
            
            # 프로그레스바 업데이트
            progress = (processed_mails / total_mails) * 100
            progress_var.set(progress)
            progress_label.config(text=f"진행률: {progress:.1f}% ({processed_mails}/{total_mails})")
            root.update()

        except Exception as e:
            logging.error(f"메일 처리 오류: {e}")

def process_folder(folder_name: str, root_name: str):
    """
    폴더와 그 하위 폴더의 메일을 처리하는 함수
    """
    global folder_key
    folder_key = folder_name

    root_dir = os.path.join(BASE_DIR, root_name)
    os.makedirs(root_dir, exist_ok=True)

    def process_subfolder(outlook_folder):
        items = outlook_folder.Items
        items.Sort("[ReceivedTime]", True)
        process_mail_items(items, root_dir)

        for subfolder in outlook_folder.Folders:
            process_subfolder(subfolder)

    root_folder = outlook.GetDefaultFolder(FOLDER_MAP[folder_key])
    process_subfolder(root_folder)

def set_date_range(months):
    """
    날짜 범위를 설정하는 함수
    """
    end_date_cal.set_date(datetime.now())
    start_date_cal.set_date(datetime.now() - timedelta(days=30*months))

def start_backup():
    """
    백업 시작 함수
    """
    global start_date, end_date, processed_mails, mail_count
    
    # 캘린더에서 선택된 날짜 가져오기
    start_date = start_date_cal.get_date()
    end_date = end_date_cal.get_date()
    
    # 날짜 범위 확인
    if start_date > end_date:
        status_label.config(text="시작 날짜가 종료 날짜보다 늦을 수 없습니다.")
        return
        
    if end_date > datetime.now().date():
        status_label.config(text="종료 날짜는 오늘 이후로 설정할 수 없습니다.")
        return
    
    # 변수 초기화
    mail_count = {"inbox": 0, "sent": 0}
    processed_mails = 0
    progress_var.set(0)  # 프로그레스바 초기화
    progress_label.config(text="")  # 진행률 레이블 초기화
    
    # 로그 시작
    logging.info(f"백업 시작")
    logging.info(f"시작 날짜: {start_date}")
    logging.info(f"종료 날짜: {end_date}")
    
    status_label.config(text="백업 진행 중...")
    root.update()
    
    start_time = time.time()  # 시작 시간 기록
    
    try:
        # 전체 메일 수 계산
        count_total_mails()
        
        if total_mails == 0:
            status_label.config(text="선택된 기간에 백업할 메일이 없습니다.")
            return
            
        process_folder("inbox", "받은편지")
        process_folder("sent", "보낸편지")
        
        end_time = time.time()  # 종료 시간 기록
        elapsed_time = end_time - start_time  # 실행 시간 계산
        
        # 결과 로깅
        logging.info(f"백업 완료")
        logging.info(f"실행 시간: {elapsed_time:.1f}초")
        logging.info(f"받은편지함 저장 개수: {mail_count['inbox']}개")
        logging.info(f"보낸편지함 저장 개수: {mail_count['sent']}개")
        logging.info(f"전체 저장 개수: {mail_count['inbox'] + mail_count['sent']}개")
        
        status_label.config(text="백업이 완료되었습니다!")
    except Exception as e:
        logging.error(f"백업 중 오류 발생: {str(e)}")
        status_label.config(text=f"오류 발생: {str(e)}")

# GUI 생성
root = tk.Tk()
root.title("아웃룩 메일 백업")
root.geometry("400x500")  # 창 크기 증가

# 프레임 생성 및 중앙 정렬
frame = ttk.Frame(root, padding="10")
frame.place(relx=0.5, rely=0.5, anchor="center")

# 날짜 범위 버튼 프레임
date_range_frame = ttk.Frame(frame)
date_range_frame.pack(pady=10)

# 날짜 범위 버튼들
ttk.Button(date_range_frame, text="최근 1개월", command=lambda: set_date_range(1)).pack(side=tk.LEFT, padx=5)
ttk.Button(date_range_frame, text="최근 3개월", command=lambda: set_date_range(3)).pack(side=tk.LEFT, padx=5)
ttk.Button(date_range_frame, text="최근 6개월", command=lambda: set_date_range(6)).pack(side=tk.LEFT, padx=5)
ttk.Button(date_range_frame, text="최근 1년", command=lambda: set_date_range(12)).pack(side=tk.LEFT, padx=5)

# 시작 날짜 선택
date_frame = ttk.Frame(frame)
date_frame.pack(pady=10)

ttk.Label(date_frame, text="시작 날짜:").pack()
start_date_cal = DateEntry(date_frame, width=15, background='darkblue',
                          foreground='white', borderwidth=2,
                          date_pattern='yyyy-mm-dd',
                          firstweekday='sunday')
start_date_cal.pack(pady=5)
start_date_cal.set_date(datetime.now() - timedelta(days=7))

# 종료 날짜 선택
ttk.Label(date_frame, text="종료 날짜:").pack()
end_date_cal = DateEntry(date_frame, width=15, background='darkblue',
                        foreground='white', borderwidth=2,
                        date_pattern='yyyy-mm-dd',
                        firstweekday='sunday')
end_date_cal.pack(pady=5)
end_date_cal.set_date(datetime.now())

# 백업 시작 버튼
backup_btn = ttk.Button(frame, text="백업 시작", command=start_backup)
backup_btn.pack(pady=20)

# 프로그레스바 추가
progress_var = tk.DoubleVar()
progress_bar = ttk.Progressbar(frame, length=300, mode='determinate', variable=progress_var)
progress_bar.pack(pady=10)

# 진행률 표시 레이블
progress_label = ttk.Label(frame, text="")
progress_label.pack(pady=5)

# 상태 표시 레이블
status_label = ttk.Label(frame, text="")
status_label.pack(pady=5)

if __name__ == "__main__":
    root.mainloop()
